https://github.com/uscensusbureau/acs-summary-file/wiki/Python-Table-Data-for-All-Tracts

According to the [data explorer notes](https://data.census.gov/table/ACSDT5Y2020.B16001?q=B16001:%20Language%20Spoken%20at%20Home%20by%20Ability%20to%20Speak%20English%20for%20the%20Population%205%20Years%20and%20Over), Public Use Microdata Sample Areas (PUMAs) (795) are the most precise available version of this dataset (for privacy reasons). More info about PUMAs [here](https://www.census.gov/programs-surveys/geography/guidance/geo-areas/pumas.htm).

In [57]:
import pandas as pd
from ftplib import FTP
import os, sys
import geopandas as gpd

In [48]:
def table_for_sumlevel(tblid, year, dataset, sumlevel):

    #create output directory. 
    outdir = 'output'
    try:
        os.mkdir(outdir)
    except FileExistsError as e:
        print(f"directory named '{outdir}' already exists. delete it and try again.")
        sys.exit(1)

    dir =f"/programs-surveys/acs/summary_file/{year}/prototype/{dataset}YRData/"

    #go to ftp site
    ftp = FTP("ftp2.census.gov")
    ftp.login("","")
    ftp.cwd(dir)

    #get .dat file based on tblid or all tables
    files = [x for x in ftp.nlst() if f"{tblid}.dat" in x or (tblid=="*" and ".dat" in x)]

    for file in files:
        #read data file and query for summary level (http faster than ftp)
        df = pd.read_csv(f"https://www2.census.gov{dir}{file}", sep="|")
        df = df[ df['GEO_ID'].str.startswith(sumlevel) ]

        #output
        if not df.empty:
            df.to_csv(f"{outdir}/{file}", sep="|", index=False)
            print(f"{outdir}/{file} output.")

[Summary level definitions](https://mcdc.missouri.edu/geography/sumlevs/#:~:text=the%20U.S.-,Summary%20Levels,-All%20MCDC%20census)

In [49]:
#get all tables for all tracts

table_for_sumlevel(tblid = 'b16001', year=2020, dataset=5, sumlevel='795')

output/acsdt5y2020-b16001.dat output.


I need to re-do this workflow and instead of saving a .dat to an output directory and re-loading it, I can directly load it as a df

In [50]:
df = pd.read_csv('output/acsdt5y2020-b16001.dat', delimiter='|')
df

,GEO_ID,B16001_E001,B16001_M001,B16001_E002,B16001_M002,B16001_E003,B16001_M003,B16001_E004,B16001_M004,B16001_E005,...,B16001_E124,B16001_M124,B16001_E125,B16001_M125,B16001_E126,B16001_M126,B16001_E127,B16001_M127,B16001_E128,B16001_M128
0,7950000US0100100,176182,664,167061,861,7098,471,3281,480,3817,...,0,29,0,29,113,117,14,23,99,113
1,7950000US0100200,186045,2183,176748,2399,5840,794,3685,633,2155,...,115,66,0,29,68,80,34,54,34,52
2,7950000US0100301,126916,2620,115383,3005,5516,793,3391,578,2125,...,22,25,0,29,32,37,18,30,14,22
3,7950000US0100302,95918,2091,88273,2217,4493,739,2129,431,2364,...,33,41,0,29,61,78,36,50,25,41
4,7950000US0100400,116264,167,106516,588,9255,552,5396,569,3859,...,23,34,0,29,0,29,0,29,0,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373,7950000US7200902,134415,604,6061,683,128238,828,21233,1347,107005,...,0,34,0,34,0,34,0,34,0,34
2374,7950000US7201001,121932,-555555555,7187,743,114680,760,22465,1558,92215,...,0,34,0,34,0,34,0,34,0,34
2375,7950000US7201002,152987,-555555555,8051,1307,144785,1297,29655,1981,115130,...,0,34,0,34,0,34,0,34,0,34
2376,7950000US7201101,113396,337,5585,779,107527,917,20893,1504,86634,...,0,34,0,34,0,34,0,34,0,34


Table header labels [here](https://www2.census.gov/programs-surveys/acs/summary_file/2020/prototype/)

In [51]:
# issue with default encoding, so I'm "replacing" the text with issues (see https://docs.python.org/3/library/codecs.html#error-handlers)
df2 = pd.read_csv('geos/ACS2020_Table_Shells.csv', encoding_errors="replace")
labels = df2[df2['Table ID'] == 'B16001']
labels

,Table ID,Line,Unique ID,Stub,Data Release
12470,B16001,,NaN,LANGUAGE SPOKEN AT HOME BY ABILITY TO SPEAK EN...,"1,5"
12471,B16001,,NaN,Universe: Population 5 years and over,NaN
12472,B16001,1,B16001_001,Total:,NaN
12473,B16001,2,B16001_002,Speak only English,NaN
12474,B16001,3,B16001_003,Spanish:,NaN
...,...,...,...,...,...
12595,B16001,124,B16001_124,"Speak English ""very well""",NaN
12596,B16001,125,B16001_125,"Speak English less than ""very well""",NaN
12597,B16001,126,B16001_126,Other and unspecified languages:,NaN
12598,B16001,127,B16001_127,"Speak English ""very well""",NaN


In [47]:
geos = pd.read_csv('Geos20205YR.csv', encoding_errors="replace")
geos

/var/folders/mf/q66l1j8d7k3d1543fzt8qwfh0000gn/T/ipykernel_72757/2272889044.py:1: DtypeWarning: Columns (3,18,32,33,34,42,43,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  geos = pd.read_csv('Geos20205YR.csv', encoding_errors="replace")


,FILEID,STUSAB,SUMLEVEL,COMPONENT,LOGRECNO,US,REGION,DIVISION,STATECE,STATE,...,PCI,BLANK.3,BLANK.4,PUMA5,DADSID,GEOID,NAME,BTTR,BTBG,BLANK.5
0,ACSSF,US,10,00,1,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0100000US,01000US,United States,NaN,NaN,NaN
1,ACSSF,US,10,01,2,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0100001US,01001US,United States -- Urban,NaN,NaN,NaN
2,ACSSF,US,10,43,3,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0100043US,01043US,United States -- Rural,NaN,NaN,NaN
3,ACSSF,US,10,89,4,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0100089US,01089US,United States -- American Indian Reservation a...,NaN,NaN,NaN
4,ACSSF,US,10,90,5,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0100090US,01090US,United States -- American Indian Reservation a...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620638,ACSSF,WY,970,0,1853,NaN,NaN,NaN,NaN,56.0,...,NaN,NaN,NaN,NaN,9700000US5606090,97000US5606090,"Weston County School District 7, Wyoming",NaN,NaN,NaN
620639,ACSSF,WY,970,0,1854,NaN,NaN,NaN,NaN,56.0,...,NaN,NaN,NaN,NaN,9700000US5606240,97000US5606240,"Washakie County School District 1, Wyoming",NaN,NaN,NaN
620640,ACSSF,WY,970,0,1855,NaN,NaN,NaN,NaN,56.0,...,NaN,NaN,NaN,NaN,9700000US5699997,97000US5699997,"School District Not Defined, Wyoming",NaN,NaN,NaN
620641,ACSSF,WY,970,0,1856,NaN,NaN,NaN,NaN,56.0,...,NaN,NaN,NaN,NaN,9700000US5699999,97000US5699999,"Remainder of Wyoming, Wyoming",NaN,NaN,NaN


In [59]:
shp = gpd.read_file('geos/cb_2020_us_puma20_500k.zip')
shp.head

<bound method NDFrame.head of      STATEFP20 PUMACE20        AFFGEOID20  GEOID20  \
0           26    03003  795P200US2603003  2603003   
1           26    03213  795P200US2603213  2603213   
2           18    03000  795P200US1803000  1803000   
3           27    00301  795P200US2700301  2700301   
4           36    00703  795P200US3600703  3600703   
...        ...      ...               ...      ...   
2482        39    00705  795P200US3900705  3900705   
2483        48    01600  795P200US4801600  4801600   
2484        39    00706  795P200US3900706  3900706   
2485        34    01405  795P200US3401405  3401405   
2486        26    02600  795P200US2602600  2602600   

                                             NAMELSAD20 STUSPS20   ST_NAME20  \
0     Macomb County (Southwest)--Sterling Heights Ci...       MI    Michigan   
1          Wayne County (Northeast)--I-94 Corridor PUMA       MI    Michigan   
2     Bartholomew & Jackson Counties--Columbus City ...       IN     Indiana   
3